In [1]:
import os
from datetime import datetime
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import pyspark.sql.types as T
from functools import reduce, partial
import re
import time

In [2]:
spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("file-streaming")
    .getOrCreate()
)
spark

In [3]:
schema = (
    T.StructType()
    .add('vip_id',T.LongType())
    .add('first_name',T.StringType())
    .add('last_name',T.StringType())
    .add('vip_type',T.StringType())
    .add('country',T.StringType())
    .add('email',T.StringType())
    .add('ExtractionDate',T.DateType())
)

In [ ]:
# => setup readstream
vip_dataframe = (
    spark.readStream.schema(schema)
    .option('maxFilesPerTrigger', 1)
    .parquet("file:///export/apps/jupyterlab/workspace/nike/data/parquet/")
)

In [ ]:
# => writing to sink
vip_activity_query = (
    vip_dataframe.writeStream
    .format('memory')
    .outputMode('append')
    .queryName('vip_latest')
    .start()
)

In [ ]:
spark.sql("select * from vip_latest").show(truncate=False)

In [ ]:
spark.streams.active[0].isActive

In [ ]:
vip_activity_query.status

In [ ]:
spark.stop()